In [1]:
from data_provider import data_factory,data_loader
import numpy as np

In [5]:
import numpy as np

In [7]:
a = [[1,2,3,4],
     [4,5,6,7]]
np.corrcoef(a)

array([[1., 1.],
       [1., 1.]])

In [11]:
a = np.array(a).T
a.shape

(4, 2)

In [2]:
import argparse
args = argparse.Namespace()
args.augmentation_ratio = 0    

In [10]:
testset = data_loader.m4Dataset_btc(args,scale=False,
        data_path='btc_t_v_withf4.csv',flag="train",size=[128,0,1])


In [9]:
testset[10][0].shape

(96, 138)

In [10]:
testset[0][1].shape

(96, 1)

In [12]:
import re
import matplotlib.pyplot as plt
import numpy as np

def extract_data_from_text(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    blocks = []
    current_block = []
    for line in lines:
        if "Args in experiment:" in line and current_block:
            blocks.append(current_block)
            current_block = []
        current_block.append(line)
    if current_block:
        blocks.append(current_block)

    block_names = []
    avg_val_ic_values = []
    avg_test_ic_values = []
    std_val_ic_values = []
    std_test_ic_values = []

    for block in blocks:
        block_text = "".join(block)
        model_match = re.search(r"Model:\s*(\S+)", block_text)

        val_ic_matches = re.findall(r"val ic:\s*([0-9.-]+)", block_text)
        test_ic_matches = re.findall(r"test ic:\s*([0-9.-]+)", block_text)

        if model_match and val_ic_matches and test_ic_matches:
            block_names.append(model_match.group(1))
            val_ic_floats = [float(value) for value in val_ic_matches]
            test_ic_floats = [float(value) for value in test_ic_matches]

            avg_val_ic_values.append(np.mean(val_ic_floats))
            avg_test_ic_values.append(np.mean(test_ic_floats))
            std_val_ic_values.append(np.std(val_ic_floats))
            std_test_ic_values.append(np.std(test_ic_floats))

    return block_names, avg_val_ic_values, avg_test_ic_values, std_val_ic_values, std_test_ic_values

def plot_ic_values(block_names, avg_val_ic_values, avg_test_ic_values, std_val_ic_values, std_test_ic_values):
    for i, name in enumerate(block_names):
        print(f'Block: {name}')
        print(f'Average val ic: {avg_val_ic_values[i]}, Std val ic: {std_val_ic_values[i]}')
        print(f'Average test ic: {avg_test_ic_values[i]}, Std test ic: {std_test_ic_values[i]}')
        print('-' * 50)

    plt.figure(figsize=(10, 6))

    for i, name in enumerate(block_names):
        plt.scatter(i, avg_val_ic_values[i], color='blue', label='val ic' if i == 0 else "")
        plt.scatter(i, avg_test_ic_values[i], color='red', label='test ic' if i == 0 else "")
        plt.text(i, avg_val_ic_values[i], f'{name}_val', fontsize=9, ha='right')
        plt.text(i, avg_test_ic_values[i], f'{name}_test', fontsize=9, ha='right')

    plt.xlabel("Block Index")
    plt.ylabel("IC Value")
    plt.title("Average val ic and test ic values for each block")
    plt.legend()
    plt.show()
file_path = 'log_overall_arch.txt'  # Replace with the actual file path

# block_names, avg_val_ic_values, avg_test_ic_values, std_val_ic_values, std_test_ic_values = extract_data_from_text(file_path)
# plot_ic_values(block_names, avg_val_ic_values, avg_test_ic_values, std_val_ic_values, std_test_ic_values)


In [1]:
from models.MLPs import Baseline_5,Baseline_10,Baseline_15,Baseline_20,Baseline_25
import torch
from collections import OrderedDict
import numpy as np
import torch
import torch.nn as nn

In [ ]:
nn.MultiheadAttention

In [ ]:

self.encoder_layer = nn.TransformerEncoderLayer(d_model=input_size, nhead=3, dim_feedforward=512)
self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=1)


In [22]:
def custom_summary(model, *inputs):
    def forward_hook(module, input, output):
        class_name = str(module.__class__).split(".")[-1].split("'")[0]
        module_idx = len(summary)

        m_key = "%s-%i" % (class_name, module_idx + 1)
        summary[m_key] = OrderedDict()
        summary[m_key]["input_shape"] = list(input[0].size())
        summary[m_key]["output_shape"] = list(output.size())
        
        params = 0
        if hasattr(module, "weight") and hasattr(module.weight, "size"):
            params += torch.prod(torch.LongTensor(list(module.weight.size())))
            summary[m_key]["trainable"] = module.weight.requires_grad
        if hasattr(module, "bias") and hasattr(module.bias, "size"):
            params += torch.prod(torch.LongTensor(list(module.bias.size())))
        summary[m_key]["nb_params"] = params

    def register_hook(module):
        if (
            not isinstance(module, nn.Sequential)
            and not isinstance(module, nn.ModuleList)
            and not (module == model)
        ):
            hooks.append(module.register_forward_hook(forward_hook))

    device = next(model.parameters()).device
    dtype = next(model.parameters()).dtype

    batch_size = inputs[0].size(0)
    summary = OrderedDict()
    hooks = []

    model.apply(register_hook)
    model(*inputs)

    for h in hooks:
        h.remove()

    print("----------------------------------------------------------------")
    print("Layer (type)               Output Shape         Param #")
    print("================================================================")
    total_params = 0
    for layer in summary:
        line_new = "{:30}  {:25} {:15}".format(
            layer,
            str(summary[layer]["output_shape"]),
            "{0:,}".format(summary[layer]["nb_params"]),
        )
        total_params += summary[layer]["nb_params"]
        print(line_new)

    print("================================================================")
    print("Total params: {0:,}".format(total_params))
    print("----------------------------------------------------------------")


In [23]:
input1 = torch.randn(1024,1, 138)
model = Baseline_5.Model()

custom_summary(model, input1, 0,0,0)


----------------------------------------------------------------
Layer (type)               Output Shape         Param #
Linear-1                        [1024, 1, 64]             8,896          
ReLU-2                          [1024, 1, 64]             0              
Linear-3                        [1024, 1, 32]             2,080          
ReLU-4                          [1024, 1, 32]             0              
Linear-5                        [1024, 1, 16]             528            
ReLU-6                          [1024, 1, 16]             0              
Linear-7                        [1024, 1, 8]              136            
ReLU-8                          [1024, 1, 8]              0              
Linear-9                        [1024, 1, 4]              36             
ReLU-10                         [1024, 1, 4]              0              
Linear-11                       [1024, 1, 2]              10             
ReLU-12                         [1024, 1, 2]              0      

In [24]:
input1 = torch.randn(2,1, 138)
model = Baseline_25.Model()

custom_summary(model, input1, 0,0,0)


----------------------------------------------------------------
Layer (type)               Output Shape         Param #
Linear-1                        [2, 1, 256]               35,584         
BatchNorm1d-2                   [2, 256, 1]               512            
ReLU-3                          [2, 1, 256]               0              
Linear-4                        [2, 1, 518]               133,126        
BatchNorm1d-5                   [2, 518, 1]               1,036          
ReLU-6                          [2, 1, 518]               0              
Linear-7                        [2, 1, 1024]              531,456        
BatchNorm1d-8                   [2, 1024, 1]              2,048          
ReLU-9                          [2, 1, 1024]              0              
Linear-10                       [2, 1, 2048]              2,099,200      
BatchNorm1d-11                  [2, 2048, 1]              4,096          
ReLU-12                         [2, 1, 2048]              0      

In [ ]:
class TransformerRegressor(nn.Module):
    def __init__(self, input_dim, output_dim, nhead, num_encoder_layers, dim_feedforward):
        super(TransformerRegressor, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=input_dim, nhead=nhead, dim_feedforward=dim_feedforward)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_encoder_layers)
        self.regressor = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        x = self.transformer_encoder(x)
        x = self.regressor(x[-1])  # use the last time step
        return x

In [4]:
import pandas as pd
import numpy as np


# Load the CSV file
csv_file = '/root/code/FinFusionNet/dataset/btc/btc_t_v_withf4ct.csv'  # Replace with your CSV file path
data = pd.read_csv(csv_file)

# Extract the 'range5' column



Pearson correlation coefficient: [ 1. nan]


In [7]:
range5 = data['range5']
# Generate a random array of the same length
# arbitrary_array = np.full(len(range5), )
mean = 0  # Replace with your desired mean
std_dev = 1  # Replace with your desired standard deviation
normal_array = np.random.normal(mean, std_dev, len(range5))

# Calculate the Pearson correlation coefficient
correlation_coefficient, _ = np.corrcoef(range5, normal_array)

# Print the result
print(f'Pearson correlation coefficient: {correlation_coefficient}')


Pearson correlation coefficient: [ 1.0000000e+00 -6.0130516e-04]
